# 🌿 Botanical Species Identification Tool v2.0\n### (e-Flora of South Africa Edition)\n\nWelcome! This tool helps you identify botanical species in South Africa by comparing your observations against data from the e-Flora of South Africa and occurrence records from GBIF.\n\n---\n\n## 🚀 How to Use This Tool (3 Simple Steps)\n\n1.  **Provide Your API Key:**\n    *   The first code cell (`CELL 1`) will run, followed by a prompt in the final cell (`CELL 3`) asking for your Google AI API key. You can get a free key from [Google AI Studio](https://aistudio.google.com/app/apikey).\n    *   **Tip:** For convenience, you can add your key to Colab's \"Secrets\" manager (click the 🔑 icon on the left) with the name `GOOGLE_API_KEY`. The tool will find it automatically.\n\n2.  **Run the Entire Notebook:**\n    *   Go to the menu and click **`Runtime` -> `Run all`**.\n    *   This will execute all the setup, data download, and function definitions automatically. It may take a minute or two.\n\n3.  **Interact with the Tool:**\n    *   Scroll down to the **very last cell (`CELL 3`)**.\n    *   Adjust the `Latitude`, `Longitude`, `Radius`, and `Taxon Name` in the interactive form.\n    *   Optionally, add your own specimen description.\n    *   The analysis will run, and the results, map, and raw data will appear below the form. You can change the parameters and the cell will re-run automatically.\n\n---\n**Data Source Credit:** This tool utilizes data from the [e-Flora of South Africa](http://www.sanbi.org.za/), managed by the South African National Biodiversity Institute (SANBI).

In [ ]:
# ==============================================================================\n# BOTANICAL SPECIES IDENTIFICATION TOOL v2.0 (e-Flora of South Africa Edition)\n# ==============================================================================\n\n# ==============================================================================\n# CELL 1: SETUP AND INSTALLATION\n# Run this cell once at the start of your session.\n# ==============================================================================\n\n# --- 1. Install necessary libraries ---\n!pip install -q -r requirements.txt\n\n# --- 2. Import all required tools ---\nimport pandas as pd\nimport google.generativeai as genai\nfrom datetime import datetime\nimport markdown\nfrom google.colab import files, userdata, drive\nfrom IPython.display import display, Markdown, HTML\nimport folium\nimport warnings\nfrom src.data_loader import download_data_files, load_dwca_data_from_local\nfrom src.core_functions import run_analysis, set_eflora_data\n\n# Ignore minor warnings to keep the output clean.\nwarnings.filterwarnings('ignore')\n\nprint("✅ Setup complete. You can now proceed to Cell 2.")\nprint(f"📅 Session started: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")

In [ ]:
# ==============================================================================\n# CELL 2: LOAD E-FLORA OF SOUTH AFRICA DATA\n# This cell automatically downloads and processes the required data files.\n# ==============================================================================\n\n# --- 1. Download the data files ---\ndownload_data_files()\n\n# --- 2. Load and Process the Data ---\nEFLORA_DATA = load_dwca_data_from_local()\nif EFLORA_DATA is not None:\n    set_eflora_data(EFLORA_DATA)\n    print("   Ready for analysis. Proceed to Cell 3.")

In [ ]:
# ==============================================================================\n# CELL 3: MAIN EXECUTION - CONFIGURE AND RUN YOUR ANALYSIS\n# This is the main control panel for the tool.\n# ==============================================================================\n\n# --- Step 1: Configure API Key ---\nprint("🔑 Configuring API access...")\ntry:\n    # Attempt to get the key from Colab Secrets\n    from google.colab import userdata\n    GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')\n    genai.configure(api_key=GOOGLE_API_KEY)\n    print("✅ API key configured successfully from Colab Secrets.\n")\nexcept (ImportError, userdata.SecretNotFoundError):\n    print("⚠️ Colab Secrets not found or key is missing.")\n    # Fallback to manual input if secrets fail\n    import getpass\n    api_key_input = getpass.getpass(\"Please enter your Google API Key: \")\n    if api_key_input:\n        GOOGLE_API_KEY = api_key_input\n        genai.configure(api_key=GOOGLE_API_KEY)\n        print("✅ API key configured manually for this session.\n")\n    else:\n        print("❌ No API key provided. Cannot proceed.")\n        raise SystemExit()\nexcept Exception as e:\n    print(f"❌ An unexpected error occurred during API configuration: {e}")\n    raise SystemExit()\n\n# --- Step 2: User Interface ---\nprint(\"=\"*60)\nprint("🌿 BOTANICAL SPECIES IDENTIFICATION TOOL (e-Flora SA)")\nprint(\"=\"*60)\n\n#@title 📍 Location Settings\nLATITUDE = -32.90539  #@param {type:\"number\"}\nLONGITUDE = 28.06159   #@param {type:\"number\"}\nRADIUS_KM = 15      #@param {type:\"slider\", min:1, max:100, step:1}\n\n#@title 🌱 Search Parameters (Try any rank: Genus, Family, Order, etc.)\nTAXON_NAME = \"Vitaceae\" #@param {type:\"string\"}\nMAX_SPECIES_TO_PROCESS = 15 #@param {type:\"slider\", min:5, max:50, step:5}\n\n#@title 🔬 Specimen Details (Optional: Leave blank for a general field guide)\nUSER_INPUT = \"Leaves palmate. Leaflets serrated, sublabous.\" #@param {type:\"string\"}\n\n# --- Step 3: Run Pre-Analysis Checks ---\nif 'EFLORA_DATA' not in globals() or EFLORA_DATA is None:\n    print("\n❌ ERROR: e-Flora data is not loaded. Please run Cell 2 successfully before this cell.")\nelse:\n    # --- Step 4: Display Search Area Map ---\n    print("\n🗺️ Visualizing Search Area...")\n    m = folium.Map(location=[LATITUDE, LONGITUDE], zoom_start=13)\n    folium.Circle(\n        location=[LATITUDE, LONGITUDE], radius=RADIUS_KM * 1000,\n        popup=f\"{RADIUS_KM}km search radius\", color='blue', fill=True, fillOpacity=0.2\n    ).add_to(m)\n    folium.Marker(\n        [LATITUDE, LONGITUDE], popup=f\"Center: {LATITUDE:.4f}, {LONGITUDE:.4f}\",\n        icon=folium.Icon(color='red')\n    ).add_to(m)\n    display(m)\n\n    # --- Step 5: Run the Full Analysis ---\n    print("\n🚀 Starting analysis... This may take a few moments.")\n    analysis_result, successful_lookups, failed_list, species_metadata = run_analysis(\n        LATITUDE, LONGITUDE, RADIUS_KM, TAXON_NAME, USER_INPUT, MAX_SPECIES_TO_PROCESS\n    )\n\n    # --- Step 6: Display the Final Results ---\n    if analysis_result:\n        print("\n\" + \"=\"*60)\n        print("📋 ANALYSIS RESULTS")\n        print(\"=\"*60)\n        display(Markdown(analysis_result))\n\n        # --- Display Collapsible Raw Data ---
        if successful_lookups:\n            html_parts = [\"<br><h3>📖 Raw e-Flora Descriptions Used in Analysis (Click to Expand)</h3>\"]\n            for item in successful_lookups:\n                description_html = markdown.markdown(item['description'])\n                html_parts.append(f\"\"\"\n                <details style=\"margin-bottom: 8px; border: 1px solid #ddd; border-radius: 4px; padding: 10px;\">\n                    <summary style=\"cursor: pointer; font-weight: bold;\">\n                        {item['name']}\n                        <span style=\"font-weight: normal; color: #555;\"> (GBIF Occurrences: {item['gbif_count']})</span>\n                    </summary>\n                    <div style=\"margin-top: 10px; padding-left: 15px; border-left: 2px solid #eee;\">\n                        {description_html}\n                    </div>\n                </details>\n                \"\"\")\n            display(HTML(\"\".join(html_parts)))\n\n        if failed_list:\n            print("\n⚠️ Could not find local descriptions for the following species:")\n            print("   \" + \", \".join(failed_list))\n    else:\n        print("\n❌ Analysis could not be completed. Please check the logs above for errors.")\n\n    print("\n\" + \"=\"*60)\n    print("✨ Process complete!")\n    print(\"=\"*60)